In [4]:
import pandas as pd
import numpy as np
from pprint import pprint


In [5]:
df = pd.read_csv('steam-200k.csv')
user_dict = dict()
game_dict = dict()
print(df.columns)

print(df.tail(2))

Index(['user_id', 'name', 'status', 'hours'], dtype='object')
          user_id  name    status  hours
199998  128470551  RUSH  purchase    1.0
199999  128470551  RUSH      play    1.4


In [10]:
df = df[(df['hours']>=2) & (df['status']=='play')]
df_filtered = df[df.groupby('name').user_id.transform(len) >= 20]
df_filtered['name'].nunique()
df_filtered.head(20)

df_filtered['user_id'] = df_filtered['user_id'].astype(str)
averages = df_filtered.groupby(['name'],as_index=False).hours.mean()
averages['avg_hours'] = averages['hours']
averages.drop('hours',axis=1,inplace=True)
# averages.head(10)
# averages.loc[averages['name']=="The Witcher 3 Wild Hunt"]
# print(float(averages[averages["name"]=="The Witcher 3 Wild Hunt"]['hours']))
# df_filtered['rating'] = df_filtered.apply(get_rating,axis=1)
# df_filtered.head(10)

# float(averages[averages['name']==df_filtered['name']]['hours'])
# df_filtered['rating'] = np.select(conditions,values)
final_ratings = pd.merge(df_filtered,averages[['name','avg_hours']],on='name')
conditions = [
    (final_ratings['hours']>=0.8*final_ratings['avg_hours']),
    (final_ratings['hours']>=0.6*final_ratings['avg_hours']) & (final_ratings['hours']<0.8*final_ratings['avg_hours']),
    (final_ratings['hours']>=0.4*final_ratings['avg_hours']) & (final_ratings['hours']<0.6*final_ratings['avg_hours']),
    (final_ratings['hours']>=0.2*final_ratings['avg_hours']) & (final_ratings['hours']<0.4*final_ratings['avg_hours']),
    final_ratings['hours']>=0,
]
values = [5,4,3,2,1]
# df_filtered['rating'] = np.select(conditions,values)
final_ratings['rating'] = np.select(conditions,values)
final_ratings

,user_id,name,status,hours,avg_hours,rating
0,151603712,The Elder Scrolls V Skyrim,play,273.0,115.351792,5
1,59945701,The Elder Scrolls V Skyrim,play,58.0,115.351792,3
2,92107940,The Elder Scrolls V Skyrim,play,110.0,115.351792,5
3,250006052,The Elder Scrolls V Skyrim,play,465.0,115.351792,5
4,11373749,The Elder Scrolls V Skyrim,play,220.0,115.351792,5
...,...,...,...,...,...,...
36540,51822361,"Warhammer 40,000 Dawn of War Soulstorm",play,23.0,14.109091,5
36541,38317154,"Warhammer 40,000 Dawn of War Soulstorm",play,5.5,14.109091,2
36542,36404933,"Warhammer 40,000 Dawn of War Soulstorm",play,5.8,14.109091,3
36543,87201181,"Warhammer 40,000 Dawn of War Soulstorm",play,24.0,14.109091,5
